In [ ]:
import pandas as pd
import numpy as np
import warnings
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import h2o

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import pickle
from h2o.automl import H2OAutoML

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.callbacks import EarlyStopping

warnings.filterwarnings('ignore')

(434237, 27)


In [ ]:
df = pd.read_csv('data_balanced.csv', encoding='utf-8')

# Analisando características com um modelo geral

### Preprocessamento one hot encode para ligas

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pickle

# Função para separar os dados em características e variável alvo
def prepare_data(df):
    X = df.drop(columns=['result', 'homeTeam', 'awayTeam', 'match_id'])
    y = df['result']
    print("Dados separados em características (X) e variável alvo (y).")
    return X, y

# Função para criar e aplicar o transformador de colunas
def create_preprocessor(X):
    numeric_features = X.select_dtypes(exclude=['object']).columns.tolist()
    categorical_features = ['league']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(drop='first'), categorical_features)
        ])
    
    print("Preprocessador criado com transformações para características numéricas e categóricas.")
    return preprocessor.fit(X)

# # Preparando os dados
X, y = prepare_data(df)
preprocessor = create_preprocessor(X)
print("Preprocessador ajustado aos dados.")

# Salvando o preprocessador para uso futuro
with open('../models/preprocessor.pickle', 'wb') as f:
    pickle.dump(preprocessor, f)
    print("Preprocessador salvo em '../models/preprocessor.pickle'.")

# Dividindo os dados em conjuntos de treino e teste
X_transformed = preprocessor.transform(X)
print("Dados transformados pelo preprocessador.")

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.20, random_state=42)
print("Dados divididos em conjuntos de treino e teste.")

Dados separados em características (X) e variável alvo (y).
Preprocessador criado com transformações para características numéricas e categóricas.
Preprocessador ajustado aos dados.
Preprocessador salvo em '../models/preprocessor.pickle'.
Dados transformados pelo preprocessador.
Dados divididos em conjuntos de treino e teste.


### Treinando modelo geral

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline

# Dicionário para armazenar histórico de treinamento e avaliação para cada liga
histories = {}

# Loop por cada liga para criar, treinar e avaliar um modelo separado
print(f"Treinando modelo geral")

model_randomf = RandomForestClassifier(n_estimators=100, random_state=42, 
                                        max_depth=10, min_samples_split=5, 
                                        min_samples_leaf=2)
# Validação cruzada
cv_scores = cross_val_score(model_randomf, X_transformed, y, cv=5, scoring='roc_auc')
print(f"Média AUC: {cv_scores.mean()}")

# Treinando e avaliando o modelo
model_randomf.fit(X_train, y_train)
train_auc = roc_auc_score(y_train, model_randomf.predict_proba(X_train)[:, 1])
test_auc = roc_auc_score(y_test, model_randomf.predict_proba(X_test)[:, 1])

feature_importance = model_randomf.feature_importances_

# Armazenando a avaliação em um dicionário
histories = {
    'cv_auc_mean': cv_scores.mean(),
    'train_auc': train_auc,
    'test_auc': test_auc,
    'feature_importance': feature_importance
}

# classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, model_randomf.predict(X_test)))


Treinando modelo geral
Média AUC: 0.5546111442010817


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, model_randomf.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62      9529
         1.0       0.60      0.60      0.60      9015

    accuracy                           0.61     18544
   macro avg       0.61      0.61      0.61     18544
weighted avg       0.61      0.61      0.61     18544

